# Pondremos a prueba las funciones requeridas para la API

## Importaciones

In [1]:
import pandas as pd

%load_ext autoreload
%autoreload 2
import utils

import warnings
warnings.filterwarnings("ignore")

## Carga de los dataset necesarios

Se cargan los conjuntos de datos necesarios para llevar a cabo las consultas solicitadas. Se decide subir los datos de esta manera, y no los dataset completos, debido a la limitada capacidad de almacenamiento que permite Render. Por otra parte, considerando que el objetivo es entregar un PVM, a modo de prueba de concepto, se consideró adecuada esta simplificación de los datasets.

Los  dataset que se utilizan son:

* **df_reviews_unido**: contiene la información relacionada a los usuarios que realizan reviews de los juegos en formato de análisis de sentimientos. Entre esta información, se encuentran las recomendaciones o no del juego por parte de usuario, la fecha del review así como datos del usuario como su id, su url del perfil.
* **df_gastos_items**: contiene la cantidad de items que consume cada usuario y el precio de cada uno de los productos que consume.
* **df_playtime_forever**: contiene para cada usuario su identificación, la url de su perfil, la cantidad de horas jugadas por cada género de juego.
* **genre_ranking**: contiene el ranking de los géneros de juegos con mas horas jugadas.
* **df_items_developer**: contiene la información relacionada con cada item de juegos como su id, precio, desarrollador y año de lanzamiento.

In [103]:
df_reviews_as = pd.read_csv('data/df_reviews_as_unido.csv')
df_gastos_items = pd.read_csv('data/df_gastos_items_unido.csv')
df_playtime_forever = pd.read_csv('data/df_playtime_forever_unido.csv')
df_items_developer = pd.read_csv('data/df_items_developer_unido.csv')

A continuación, se desarrollan cada una de las funciones solicitadas.

## developer

Esta función recibe como parámetro 'developer', que es la empresa desarrolladora del juego, y devuelve la cantidad de items que desarrolla dicha empresa y el porcentaje de contenido Free por año por sobre el total que desarrolla

In [12]:
def developer(desarrollador):
    # Filtra el dataframe por desarrollador de interés
    data_filtrada = df_items_developer[df_items_developer['developer'] == desarrollador]
    # Calcula la cantidad de items por año
    cantidad_por_año = data_filtrada.groupby('release_anio')['item_id'].count()
    # Calcula la cantidad de elementos gratis por año
    cantidad_gratis_por_año = data_filtrada[data_filtrada['price'] == 0.0].groupby('release_anio')['item_id'].count()
    # Calcula el porcentaje de elementos gratis por año
    porcentaje_gratis_por_año = (cantidad_gratis_por_año / cantidad_por_año * 100).fillna(0).astype(int)

    result_dict = {
        'cantidad_por_año': cantidad_por_año.to_dict(),
        'porcentaje_gratis_por_año': porcentaje_gratis_por_año.to_dict()
    }
    
    return result_dict

In [13]:
desarrollador = 'Mechanical Boss'
developer(desarrollador)

{'cantidad_por_año': {'2016': 2, '2017': 1},
 'porcentaje_gratis_por_año': {'2016': 50, '2017': 0}}

## userdata

Esta función tiene por parámentro 'user_id' y devulve la cantidad de dinero gastado por el usuario, el porcentaje de recomendaciones que realizó sobre la cantidad de reviews que se analizan y la cantidad de items que consume el mismo.

In [14]:
def userdata(user_id):
    
    # Filtra por el usuario de interés
    usuario = df_reviews_as[df_reviews_as['user_id'] == user_id]
    # Calcula la cantidad de dinero gastado para el usuario de interés
    cantidad_dinero = df_gastos_items[df_gastos_items['user_id']== user_id]['price'].iloc[0]
    # Busca el count_item para el usuario de interés    
    count_items = df_gastos_items[df_gastos_items['user_id']== user_id]['items_count'].iloc[0]
    
    # Calcula el total de recomendaciones realizadas por el usuario de interés
    total_recomendaciones = usuario['reviews_recommend'].sum()
    # Calcula el total de reviews realizada por todos los usuarios
    total_reviews = len(df_reviews_as['user_id'].unique())
    # Calcula el porcentaje de recomendaciones realizadas por el usuario de interés
    porcentaje_recomendaciones = (total_recomendaciones / total_reviews) * 100
    
    return {
        'cantidad_dinero': cantidad_dinero,
        'porcentaje_recomendacion': round(porcentaje_recomendaciones, 2),
        'total_items': count_items.astype(int)
    }

In [15]:
user_id = 'EchoXSilence'
userdata(user_id)

{'cantidad_dinero': 189.84,
 'porcentaje_recomendacion': 0.03,
 'total_items': 23}

## userforgenre

Esta función recibe como parámetro el género de un videojuego y devuelve el top 5 de los usuarios con más horas de juego en el género ingresado, indicando el id del usuario y el url de su perfil.

In [16]:
def userforgenre(genero):
    # Filtra el dataframe por el género de interés
    data_por_genero = df_playtime_forever[df_playtime_forever['genres'] == genero]
    # Agrupa el dataframe filtrado por usuario y suma la cantidad de horas
    top_users = data_por_genero.groupby(['user_url', 'user_id'])['playtime_horas'].sum().nlargest(5).reset_index()
    
    # Se hace un diccionario vacío para guardar los datos que se necesitan
    top_users_dict = {}
    for index, row in top_users.iterrows():
        # User info recorre cada fila del top 5 y lo guarda en el diccionario
        user_info = {
            'user_id': row['user_id'],
            'user_url': row['user_url']
        }
        top_users_dict[index + 1] = user_info
    
    return top_users_dict

In [17]:
genero = 'Action'
userforgenre(genero)

{1: {'user_id': 'Sp3ctre', 'user_url': 'http://steamcommunity.com/id/Sp3ctre'},
 2: {'user_id': 'shinomegami',
  'user_url': 'http://steamcommunity.com/id/shinomegami'},
 3: {'user_id': 'REBAS_AS_F-T',
  'user_url': 'http://steamcommunity.com/id/REBAS_AS_F-T'},
 4: {'user_id': 'Terminally-Chill',
  'user_url': 'http://steamcommunity.com/id/Terminally-Chill'},
 5: {'user_id': 'DownSyndromeKid',
  'user_url': 'http://steamcommunity.com/id/DownSyndromeKid'}}

## best_developer_year

 Devuelve el top 3 de desarrolladores con juegos MÁS recomendados por usuarios para el año dado. (reviews.recommend = True y comentarios positivos)
Ejemplo de retorno: [{"Puesto 1" : X}, {"Puesto 2" : Y},{"Puesto 3" : Z}]

In [70]:
df_reviews_as

,user_id,user_url,reviews_item_id,reviews_helpful,reviews_recommend,reviews_date,sentiment_analysis,developer,release_anio
0,76561197970982479,http://steamcommunity.com/profiles/76561197970...,1250,No ratings yet,True,2011-11-05,1,Tripwire Interactive,2009
1,EndAtHallow,http://steamcommunity.com/id/EndAtHallow,1250,No ratings yet,True,2015-01-15,1,Tripwire Interactive,2009
2,76561198077432581,http://steamcommunity.com/profiles/76561198077...,1250,No ratings yet,True,2014-12-12,1,Tripwire Interactive,2009
3,76561198057958244,http://steamcommunity.com/profiles/76561198057...,1250,0 of 1 people (0%) found this review helpful,True,2013-12-13,0,Tripwire Interactive,2009
4,46366536564574576346346546,http://steamcommunity.com/id/46366536564574576...,1250,2 of 3 people (67%) found this review helpful,True,2014-08-19,1,Tripwire Interactive,2009
...,...,...,...,...,...,...,...,...,...
48797,ButtBurger2,http://steamcommunity.com/id/ButtBurger2,73010,No ratings yet,True,2012-05-17,0,Colossal Order Ltd.,2011
48798,76561198064526566,http://steamcommunity.com/profiles/76561198064...,378930,3 of 17 people (18%) found this review helpful,False,Formato inválido,1,Skyjaz Games,2016
48799,haungaraho,http://steamcommunity.com/id/haungaraho,16600,No ratings yet,True,2012-10-22,2,Redlynx,2008
48800,UnseenPrecision,http://steamcommunity.com/id/UnseenPrecision,232950,No ratings yet,True,2014-01-19,1,Halycon Media GmbH &amp; Co. KG,2013


In [75]:
# Reemplazar los valores 'Dato no disponible' con cero
df_reviews_as['release_anio'] = df_reviews_as['release_anio'].replace('Dato no disponible', 0)

# Convertir la columna 'count' a tipo entero
df_reviews_as['release_anio'] = df_reviews_as['release_anio'].astype(int)


In [76]:
def best_developer_year(año: int):
    # Filtrar las reseñas para el año dado y donde las recomendaciones sean verdaderas y los comentarios sean positivos
    df_filtered = df_reviews_as[(df_reviews_as['release_anio'] == año) & (df_reviews_as['reviews_recommend'] == True) & (df_reviews_as['sentiment_analysis'] == 2)]
    
    # Contar el número de juegos recomendados para cada desarrollador
    developer_counts = df_filtered['developer'].value_counts().reset_index(name='count')
    
    # Tomar los tres primeros desarrolladores
    top_developers = developer_counts.head(3)
    
    # Crear la lista en el formato requerido
    resultado = [{"Puesto {}".format(i+1): row['developer']} for i, row in top_developers.iterrows()]
    
    return resultado





In [77]:
año = 1998
best_developer_year(año)

[{'Puesto 1': 'Valve'},
 {'Puesto 2': 'Oddworld Inhabitants'},
 {'Puesto 3': 'Epic Games, Inc.'}]

## sentiment_analysis

Esta función recibe como parámetro el año de lanzamiento de un juego y según ese año devuelve una lista con la cantidad de registros de reseñas de usuarios que se encuentren categorizados con un análisis de sentimiento, como Negativo, Neutral y Positivo.

In [114]:
def developer_reviews_analysis(desarrolladora: str):
    # Filtrar las reseñas para el desarrollador dado
    reviews_developer = df_reviews_as[df_reviews_as['developer'] == desarrolladora]
    
    # Contar los registros de reseñas categorizadas como positivas y negativas
    positive_reviews_count = len(reviews_developer[reviews_developer['sentiment_analysis'] == 2])
    neutral_reviews_count = len(reviews_developer[reviews_developer['sentiment_analysis'] == 1])
    negative_reviews_count = len(reviews_developer[reviews_developer['sentiment_analysis'] == 0])
    
    # Crear el diccionario de análisis de reseñas del desarrollador
    developer_reviews_dict = {desarrolladora: {'Positive': positive_reviews_count, 'Neutro': neutral_reviews_count, 'Negative': negative_reviews_count}}
    
    return developer_reviews_dict


In [115]:
desarrollador = 'Colossal Order Ltd.'
developer_reviews_analysis(desarrollador)

{'Colossal Order Ltd.': {'Positive': 20, 'Neutro': 54, 'Negative': 10}}